In [10]:
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime
import pytz
import copy
import datetime as datetime

# Getting data functions and dependencies

In [11]:
def get_data(s_name):
    
    s = pd.read_csv(f"../Data_Updated/Stocks_-5/{s_name}.csv")
    s=s[s['Date'] >= '2019-01-01']
#     s=s[s['Date'] <= '2020-01-01']
    
    s=s[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    s["Signal"] = np.zeros(len(s))
    s["Running Status"] = np.zeros(len(s))
    s["Target"] = np.zeros(len(s))
    s["Stop_Loss"] = np.zeros(len(s))
    s["P&L"] = np.zeros(len(s))
    s["Cum_Returns"] = np.ones(len(s))
    s["Perc_Change"] = np.zeros(len(s))
    
    return s

# Backtesting functions

In [12]:
def WILLIAMS_R(DF, n=14):
    
    df = DF.copy()
    df['Highest_High'] = DF['High'].rolling(window=n).max()
    df['Lowest_Low'] = DF['Low'].rolling(window=n).min()
    df['Williams_%R'] = (df['Highest_High'] - df['Close']) / (df['Highest_High'] - df['Lowest_Low']) * -100
    return df['Williams_%R']

def Bollinger_Bands (DF, n=20,std=2):
    
    df=DF.copy()
    df["ML"] = df["Close"].rolling(n).mean()
    df["UB"] = df["ML"] + std*df["Close"].rolling(n).std(ddof=0)
    df["LB"] = df["ML"] - std*df["Close"].rolling(n).std(ddof=0)
#     df["BW"] = df["UB"] - df["LB"]
    return df[["ML","UB","LB"]]

def SMA(DF, n):
    df = DF.copy()
    df[f'{n}SMA'] = df['Close'].rolling(window=n).mean()
    return df[f'{n}SMA']

In [13]:
def round_to_25(x):
    return np.round(x * 20) / 20 if not np.isnan(x) else np.nan

In [14]:
def set_stop_loss_and_target(df,sl,tg):
    
    # Set Stop_Loss
    has_signal = df['Signal'] == 1
    df.loc[has_signal, 'Stop_Loss'] = round(df.loc[has_signal, 'Close']*(1-sl),2)
    has_signal = df['Signal'] == -1    
    df.loc[has_signal, 'Stop_Loss'] = round(df.loc[has_signal, 'Close'] * (1+sl),2)

    # Set Target
    long_condition = df['Signal'] == 1
    short_condition = df['Signal'] == -1

    df.loc[long_condition, 'Target'] = round(df['Close']*(1+tg),2)
    df.loc[short_condition, 'Target'] = round(df['Close']*(1-tg),2)
    return df


In [15]:
def CAGR(DF):
    year_range = 252
    df=DF.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    ia = df.index.to_numpy()
    
    sd= df.loc[ia[1],"Date"]
    ed= df.loc[ia[-1],"Date"]
    x=ed-sd
    x=x.days
    numberofyears=x/year_range
#     print(numberofyears)
    CAGR = (df.loc[ia[-1],"Cum_Returns"])**(1/numberofyears)-1
    return CAGR

def volatility(DF):
    df = DF.copy()
    vol = df["Perc_Change"].std() * np.sqrt(252)
    return vol

def Sharpe(DF,RR=0.05):
    df=DF.copy()
    Sharpe = (CAGR(df)-RR)/volatility(df)
    return Sharpe

def max_dd(DF):
    df = DF.copy()
    df["cum_roll_max"] = df["Cum_Returns"].cummax()
    df["drawdown"] = df["cum_roll_max"] - df["Cum_Returns"]
    return (df["drawdown"]/df["cum_roll_max"]).max()*100

def calmar(DF):
    df = DF.copy()
    return CAGR(df)/max_dd(df)*100

In [16]:
from datetime import time

def procedure(df):
    df['WRSI'] = WILLIAMS_R(df,14)
    df['4SMA'] = SMA(df,4)
    df['9SMA'] = SMA(df,9)
    df[["ML","UB","LB"]] = Bollinger_Bands(df)
    
    df.isnull().sum()
    df= df[20:]
    df.reset_index(inplace=True)
    
    # setting buy sell conditions
    w_rsi_positive_slope = df['WRSI'] > df['WRSI'].shift(1)
    w_rsi_negative_slope = df['WRSI'] < df['WRSI'].shift(1)

    conditions = [(0.97 * df.Close<=df.LB) & (df['4SMA']*1.01>df['9SMA']) & (df.WRSI<-70), 
                  (1.03 * df.Close>=df.UB) & (df['4SMA']<df['9SMA']*1.01) & (df.WRSI>-30)]
    choices = [1,-1]
    df["Signal"] = np.select(conditions,choices)
    
    sl = 0.1
    tg = 0.4
    df = set_stop_loss_and_target(df,sl,tg)
    df["Stop_Loss"] = df["Stop_Loss"].apply(round_to_25)
    df["Target"] = df["Target"].apply(round_to_25)
    

#     starting_range = time(9, 10, 0)  
#     ending_range = time(21,15,0)
    # mask = (df["Time"] >= starting_range) & (df["Time"] <= ending_range)
    # df = df[mask]
    df = df.iloc[:,1:]
    #time mask
    
    df.reset_index(drop=True, inplace=True)

    rtg=0.02

    ia = df.index.to_numpy()
    for x in ia:


        try:
            if df.loc[x,"Signal"]!=0 and df.loc[x,"Running Status"] ==0:
                df.loc[x,"Running Status"] =1
                idx = x
                na = ia[idx:]
                rtg_n=0


                for i in na[1:]:

                    if df.loc[x,"Signal"]==1:

                        if df.loc[i,"Low"]< df.loc[x,"Stop_Loss"] :
                            df.loc[i,"Running Status"] = -1
                            break

                        elif rtg_n >= 3:
                            df.loc[x,"Stop_Loss"] = df.loc[i,"Low"]
                            df.loc[i,"Running Status"] = 1
                            rtg_n=rtg_n+1
                            continue


                        elif df.loc[i,"High"]> df.loc[x,"Target"]:
                            df.loc[i,"Running Status"] = 1
                            df.loc[x,"Target"] = (1+rtg)* df.loc[x,"Target"]
                            df.loc[x,"Stop_Loss"] = (1+rtg*1.2) * df.loc[x,"Stop_Loss"] 
                            rtg_n =rtg_n+1

                            df.loc[x,"Stop_Loss"] = round_to_25(df.loc[x,"Stop_Loss"])
                            df.loc[x,"Target"] = round_to_25(df.loc[x,"Target"])
                            continue


                        else:
                            df.loc[i,"Running Status"] = 1

                    else:

                        if df.loc[i,"High"]>df.loc[x,"Stop_Loss"]:
                            df.loc[i,"Running Status"] = -1
                            break

                        elif rtg_n >= 3:
                            df.loc[x,"Stop_Loss"] = df.loc[i,"High"]
                            df.loc[i,"Running Status"] = 1
                            rtg_n=rtg_n+1
                            continue

                        elif df.loc[i,"Low"]< df.loc[x,"Target"]:
                            df.loc[i,"Running Status"] = 1
                            df.loc[x,"Target"] = (1-rtg) * df.loc[x,"Target"]
                            df.loc[x,"Stop_Loss"] = (1-rtg*1.2) * df.loc[x,"Stop_Loss"] 
                            rtg_n=rtg_n+1

                            df.loc[x,"Stop_Loss"] = round_to_25(df.loc[x,"Stop_Loss"])
                            df.loc[x,"Target"] = round_to_25(df.loc[x,"Target"])
                            continue

                        else:
                            df.loc[i,"Running Status"] = 1

        except KeyError as e:
            print(f"Error: {e}")


    #above code maintains the running status
    #yeah finally works!
    
    #defining the P&L

    amount_initial=100000
    # slippage = 
    amount=amount_initial
    i=0
    while (i<=ia.max()):

        try:
            df.loc[i,"Cum_Returns"]=df.loc[i-1,"Cum_Returns"]
        except:
            df.loc[i,"Cum_Returns"] = 1

        if (df.loc[i,"Cum_Returns"]) <0:
            idx = i
            na = ia[idx:]    
            for x in na:
                df.loc[x,"Cum_Returns"] = 0

            break

        try:

            quantity = amount//df.loc[i,"Close"]

            if (df.loc[i,"Signal"]!=0 and i==0) or (df.loc[i,"Signal"]!=0 and (i!=0 and df.loc[i-1,"Running Status"] !=1)):

                idx = i

                na = ia[idx:]


                for x in na:
#                     print(i,x)

                    try:
                        df.loc[x,"Cum_Returns"]=df.loc[x-1,"Cum_Returns"]
                    except:
                        df.loc[x,"Cum_Returns"] = 1

                    if df.loc[x,"Running Status"]==-1:

                        if df.loc[i,"Signal"]==1:

                            if df.loc[x,"High"]>df.loc[i,"Target"]:
    #                             as we exit at SL so this condition never really occurs, we are using trailing now
                                df.loc[x,"P&L"] = (df.loc[i,"Stop_Loss"] - df.loc[i,"Close"])*quantity
                                sell = df.loc[i,"Target"]*quantity

#                             elif df.loc[x,"Time"]==ending_range:
#                                 df.loc[x,"P&L"] = (df.loc[x,"Close"] - df.loc[i,"Close"])*quantity
#                                 sell = df.loc[x,"Close"] * quantity

                            else:
                                df.loc[x,"P&L"] = (df.loc[i,"Stop_Loss"] - df.loc[i,"Close"])*quantity
                                sell = df.loc[i,"Stop_Loss"] * quantity


                        else:

                            if df.loc[x,"Low"]<df.loc[i,"Target"]:
    #                             would never occur
                                df.loc[x,"P&L"] = (df.loc[i,"Close"] - df.loc[i,"Target"])*quantity
                                sell =  df.loc[i,"Target"]*quantity

#                             elif df.loc[x,"Time"]==ending_range:
#                                 df.loc[x,"P&L"] = (df.loc[i,"Close"] - df.loc[x,"Close"])*quantity
#                                 sell = df.loc[x,"Close"]*quantity

                            else:
                                df.loc[x,"P&L"] = (df.loc[i,"Stop_Loss"] - df.loc[i,"Close"] )*quantity
                                sell = df.loc[i,"Stop_Loss"]*quantity

                # taxes

                        if (df.loc[x,"Date"] == df.loc[i,"Date"]) :
                                df.loc[x,"P&L"] = df.loc[x,"P&L"] - 40 - 0.0004*(sell)

                        else:
                            df.loc[x,"P&L"] = df.loc[x,"P&L"] - 0.0022*(sell)

                        amount_prev=amount
                        amount= amount+df.loc[x,"P&L"]
                        df.loc[x,"Perc_Change"]=(amount-amount_prev)/amount_prev
                        df.loc[x,"Cum_Returns"]=amount/amount_initial

                        i=x

                        break
            i=i+1




        except KeyError as e:
            print(f"Error: {e}")
            i=i+1

    #yeah finally works!!
    
    win_rate = len(df[df["P&L"] > 0])/(len(df[df["P&L"] > 0])+len(df[df["P&L"] < 0]))*100
    returnx = amount/amount_initial
    num_trades= len(df[df["P&L"]!=0])
    cagr = CAGR(df)*100
    vol = volatility(df)*100
    sh = Sharpe(df)
    dd = max_dd(df)
    cal = calmar(df)
    actual_ret = df.loc[len(df)-1,"Close"]/df.loc[0,"Close"]
    
    return win_rate, returnx,actual_ret,num_trades,cagr,vol,sh,dd,cal
    


# Getting all stock list and defining base df

In [17]:
nifty500data = pd.read_csv("../Data_Updated/nifty500-5.csv")
ticker = []
for i in nifty500data.symbol:
    ticker.append(i)
    
ticker

['ADANIPORTS',
 'ASIANPAINT',
 'AXISBANK',
 'BAJAJ-AUTO',
 'BAJFINANCE',
 'BAJAJFINSV',
 'BPCL',
 'BHARTIARTL',
 'INFRATEL',
 'CIPLA',
 'COALINDIA',
 'DRREDDY',
 'EICHERMOT',
 'GAIL',
 'GRASIM',
 'HCLTECH',
 'HDFCBANK',
 'HEROMOTOCO',
 'HINDALCO',
 'HINDPETRO',
 'HINDUNILVR',
 'HDFC',
 'ITC',
 'ICICIBANK',
 'IBULHSGFIN',
 'IOC',
 'INDUSINDBK',
 'INFY',
 'JSWSTEEL',
 'KOTAKBANK',
 'LT',
 'M&M',
 'MARUTI',
 'NTPC',
 'ONGC',
 'POWERGRID',
 'RELIANCE',
 'SBIN',
 'SUNPHARMA',
 'TCS',
 'TATAMOTORS',
 'TATASTEEL',
 'TECHM',
 'TITAN',
 'UPL',
 'ULTRACEMCO',
 'VEDL',
 'WIPRO',
 'YESBANK',
 'ZEEL',
 'ABB',
 'ACC',
 'ABCAPITAL',
 'AMBUJACEM',
 'ASHOKLEY',
 'AUROPHARMA',
 'DMART',
 'BANDHANBNK',
 'BANKBARODA',
 'BEL',
 'BHEL',
 'BIOCON',
 'BOSCHLTD',
 'BRITANNIA',
 'CADILAHC',
 'COLPAL',
 'CONCOR',
 'DLF',
 'DABUR',
 'GICRE',
 'GODREJCP',
 'HDFCLIFE',
 'HAVELLS',
 'HINDZINC',
 'ICICIGI',
 'ICICIPRULI',
 'INDIGO',
 'L&TFH',
 'LICHSGFIN',
 'LUPIN',
 'MRF',
 'MARICO',
 'MOTHERSUMI',
 'NHPC',
 'NMDC',

In [ ]:
final = pd.DataFrame(columns=["Symbol","WinRate","Returns","Actual_Ret","No.","CAGR","Vol","Sharpe","Drawdown","Calmar"])
final.to_csv("5yearagoresults_4:1.csv", index=False)

def process_tickers(ticker_list):
    # Initialize the list for tickers that cause exceptions
    exception_list = []

    for symbol in ticker_list:
        try:
            print(symbol)
            # Read the CSV file without creating an extra index column
            final = pd.read_csv("5yearagoresults_4:1.csv")
            df = get_data(symbol)
            win_rate, returnx,actual_ret, num_trades, cagr, vol, sh, dd, cal = procedure(df)

            # Create a new DataFrame for the current ticker's data
            new_row = pd.DataFrame({
                "Symbol": [symbol],
                "WinRate": [win_rate],
                "Returns": [returnx],
                "Actual_Ret" : [actual_ret],
                "No.": [num_trades],
                "CAGR": [cagr],
                "Vol": [vol],
                "Sharpe": [sh],
                "Drawdown": [dd],
                "Calmar": [cal]
            })

            # Concatenate the new row to the final dataframe
            final = pd.concat([final, new_row], ignore_index=True)
            
            # Save the DataFrame to the CSV file
            final.to_csv("5yearagoresults_4:1.csv", index=False)
        except Exception as e:
            print(f"Exception {e}, {symbol}")
            # Add the ticker to the exception list
            exception_list.append(symbol)

    if len(exception_list)!=0:
        # Recursively process the tickers that caused exceptions
        process_tickers(exception_list)
    else:
        # Print the list of tickers that caused exceptions
        print("All tickers processed successfully.")

# Start the recursive processing with the initial list of tickers
process_tickers(ticker)


ADANIPORTS


/var/folders/7j/_shk7kkj6054qdpks42tbwxc0000gn/T/ipykernel_44145/608505689.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Signal"] = np.select(conditions,choices)
/var/folders/7j/_shk7kkj6054qdpks42tbwxc0000gn/T/ipykernel_44145/608505689.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Stop_Loss"] = df["Stop_Loss"].apply(round_to_25)
/var/folders/7j/_shk7kkj6054qdpks42tbwxc0000gn/T/ipykernel_44145/608505689.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

ASIANPAINT
AXISBANK


/var/folders/7j/_shk7kkj6054qdpks42tbwxc0000gn/T/ipykernel_44145/608505689.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Signal"] = np.select(conditions,choices)
/var/folders/7j/_shk7kkj6054qdpks42tbwxc0000gn/T/ipykernel_44145/608505689.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Stop_Loss"] = df["Stop_Loss"].apply(round_to_25)
/var/folders/7j/_shk7kkj6054qdpks42tbwxc0000gn/T/ipykernel_44145/608505689.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

BAJAJ-AUTO


...

Exception zero-size array to reduction operation maximum which has no identity, EXCELCROP
GDL
Exception zero-size array to reduction operation maximum which has no identity, GDL
GUJFLUORO
Exception zero-size array to reduction operation maximum which has no identity, GUJFLUORO
HSIL
Exception zero-size array to reduction operation maximum which has no identity, HSIL
KALPATPOWR
Exception zero-size array to reduction operation maximum which has no identity, KALPATPOWR
LAKSHVILAS
Exception zero-size array to reduction operation maximum which has no identity, LAKSHVILAS
MAGMA
Exception zero-size array to reduction operation maximum which has no identity, MAGMA
MAXINDIA
Exception zero-size array to reduction operation maximum which has no identity, MAXINDIA
MERCK
Exception zero-size array to reduction operation maximum which has no identity, MERCK
MINDAIND
Exception zero-size array to reduction operation maximum which has no identity, MINDAIND
MONSANTO
Exception zero-size array to reduction 

Exception zero-size array to reduction operation maximum which has no identity, LTI
MAHINDCIE
Exception zero-size array to reduction operation maximum which has no identity, MAHINDCIE
MINDTREE
Exception zero-size array to reduction operation maximum which has no identity, MINDTREE
RNAM
Exception zero-size array to reduction operation maximum which has no identity, RNAM
SHRIRAMCIT
Exception zero-size array to reduction operation maximum which has no identity, SHRIRAMCIT
STRTECH
Exception zero-size array to reduction operation maximum which has no identity, STRTECH
TIFIN
Exception zero-size array to reduction operation maximum which has no identity, TIFIN
TATAGLOBAL
Exception zero-size array to reduction operation maximum which has no identity, TATAGLOBAL
WABCOINDIA
Exception zero-size array to reduction operation maximum which has no identity, WABCOINDIA
8KMILES
Exception zero-size array to reduction operation maximum which has no identity, 8KMILES
ALBK
Exception zero-size array to redu

Exception zero-size array to reduction operation maximum which has no identity, INFRATEL
HDFC
Exception zero-size array to reduction operation maximum which has no identity, HDFC
CADILAHC
Exception zero-size array to reduction operation maximum which has no identity, CADILAHC
MOTHERSUMI
Exception zero-size array to reduction operation maximum which has no identity, MOTHERSUMI
SRTRANSFIN
Exception zero-size array to reduction operation maximum which has no identity, SRTRANSFIN
ADANITRANS
Exception zero-size array to reduction operation maximum which has no identity, ADANITRANS
AMARAJABAT
Exception zero-size array to reduction operation maximum which has no identity, AMARAJABAT
BHARATFIN
Exception zero-size array to reduction operation maximum which has no identity, BHARATFIN
DHFL
Exception zero-size array to reduction operation maximum which has no identity, DHFL
GSKCONS
Exception zero-size array to reduction operation maximum which has no identity, GSKCONS
GRUH
Exception zero-size arra

Exception zero-size array to reduction operation maximum which has no identity, MERCK
MINDAIND
Exception zero-size array to reduction operation maximum which has no identity, MINDAIND
MONSANTO
Exception zero-size array to reduction operation maximum which has no identity, MONSANTO
NIITTECH
Exception zero-size array to reduction operation maximum which has no identity, NIITTECH
NBVENTURES
Exception zero-size array to reduction operation maximum which has no identity, NBVENTURES
ORIENTBANK
Exception zero-size array to reduction operation maximum which has no identity, ORIENTBANK
PVR
Exception zero-size array to reduction operation maximum which has no identity, PVR
PHILIPCARB
Exception zero-size array to reduction operation maximum which has no identity, PHILIPCARB
SREINFRA
Exception zero-size array to reduction operation maximum which has no identity, SREINFRA
SYNDIBANK
Exception zero-size array to reduction operation maximum which has no identity, SYNDIBANK
TATACOFFEE
Exception zero-si

Exception zero-size array to reduction operation maximum which has no identity, ALBK
ANDHRABANK
Exception zero-size array to reduction operation maximum which has no identity, ANDHRABANK
BAJAJCORP
Exception zero-size array to reduction operation maximum which has no identity, BAJAJCORP
CAPF
Exception zero-size array to reduction operation maximum which has no identity, CAPF
DENABANK
Exception zero-size array to reduction operation maximum which has no identity, DENABANK
EXCELCROP
Exception zero-size array to reduction operation maximum which has no identity, EXCELCROP
GDL
Exception zero-size array to reduction operation maximum which has no identity, GDL
GUJFLUORO
Exception zero-size array to reduction operation maximum which has no identity, GUJFLUORO
HSIL
Exception zero-size array to reduction operation maximum which has no identity, HSIL
KALPATPOWR
Exception zero-size array to reduction operation maximum which has no identity, KALPATPOWR
LAKSHVILAS
Exception zero-size array to reduc